In [46]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import illustris_python_mod as il
import sys
import os
sys.path.append('../BH_dynamics_analysis')
sys.path.append('/home/pranavsatheesh/arepo_package/')
import arepo_package as arepo

from tqdm import tqdm

In [47]:
path_to_output='/orange/lblecha/aklantbhowmick/GAS_BASED_SEED_MODEL_UNIFORM_RUNS//L12p5n512/AREPO/output_ratio10_SFMFGM5_seed5.00_bFOF_LW10_DFD_3/' # this is the folder containing the simulation run
basePath=path_to_output

merger_properties=np.load(basePath + '/merger_statistics_subhalo_remnants.npy',allow_pickle=True)[0]
if (1==1):
    merger_type=np.array(merger_properties['merger_type'])
    merger_redshift=np.array(merger_properties['merger_redshift'])
    BH_ID1=np.array(merger_properties['BH_ID1'])
    BH_ID2=np.array(merger_properties['BH_ID2'])
    BH_Mass1=np.array(merger_properties['BH_Mass1'])
    BH_Mass2=np.array(merger_properties['BH_Mass2'])
    remnant_redshift=np.array(merger_properties['remnant_redshift'])
    remnant_SubhaloStellarMass=np.array(merger_properties['remnant_SubhaloStellarMass'])
    remnant_SubhaloDarkMatterMass=np.array(merger_properties['remnant_SubhaloDarkMatterMass'])
    remnant_SubhaloHalfMassRadii=np.array(merger_properties['remnant_SubhaloHalfMassRadii'])
    remnant_central_or_satellite=np.array(merger_properties['remnant_central_or_satellite'])
    remnant_SubhaloID=np.array(merger_properties['remnant_SubhaloID'])

In [48]:
merger_indices = np.arange(0,len(merger_redshift))

In [49]:
tree=h5py.File(basePath+'/postprocessing/tree_extended.hdf5')
SnapNum=tree.get('SnapNum')[:]
NextProgenitorID=tree.get('NextProgenitorID')[:]
FirstProgenitorID=tree.get('FirstProgenitorID')[:]
SubfindID=tree.get('SubfindID')[:]
SubhaloID = tree.get('SubhaloID')[:]
list(tree.keys())

['DescendantID',
 'FirstProgenitorID',
 'FirstSubhaloInFOFGroupID',
 'LastProgenitorID',
 'MainLeafProgenitorID',
 'Mass',
 'MassHistory',
 'NextProgenitorID',
 'NextSubhaloInFOFGroupID',
 'NumParticles',
 'RootDescendantID',
 'SnapNum',
 'SubfindID',
 'SubhaloID',
 'SubhaloIDRaw',
 'TreeID']

In [50]:
# inn = 1
# o,remnant_snap=arepo.desired_redshift_to_output_redshift(basePath,remnant_redshift[inn])
# mask_snap = SnapNum == remnant_snap
# mask_subfindID = SubfindID == remnant_SubhaloID[inn]
# mask = mask_snap& mask_subfindID

#fpID = FirstProgenitorID[mask][0]
#np.where(SubhaloID==fpID)[0][0]

In [51]:
def get_sublink_info_for_merger_event(merger_index):
    o,remnant_snap=arepo.desired_redshift_to_output_redshift(basePath,remnant_redshift[merger_index])
    mask_snap = SnapNum == remnant_snap
    mask_subfindID = SubfindID == remnant_SubhaloID[merger_index]
    mask = mask_snap & mask_subfindID

    try:
        fpID = FirstProgenitorID[mask][0]
        #The first progenitor
    except:
        # If the mask does not return any results, it means the subhalo is not found in this snapshot
        return -1, -1, -1, -1, remnant_snap

    if fpID == -1:
        #First progenitor ID is -1, which means this remnant subhalo and has no progenitors
        return -1, -1, -1, -1, remnant_snap
    
    else:
        #The other progenitor is found by matching the subhalo ID to the fpID of this subhalo and extracting the nextprogenitor ID
        npID = NextProgenitorID[np.where(SubhaloID==fpID)[0][0]]
        dID = SubhaloID[mask][0]
        subfind_ID = SubfindID[mask][0]

        return subfind_ID, fpID, npID, dID, remnant_snap

In [52]:
merger_indices

array([   0,    1,    2, ..., 3139, 3140, 3141])

In [53]:
run_name='output_ratio10_SFMFGM5_seed5.00_bFOF_LW10_DFD_3'
save_path='/home/pranavsatheesh/host_galaxies/data/'
brahma_tree_connect_file=save_path+"tree_subhalo_bh_connect_data_run_%s.npz"%(run_name)

In [54]:
brahma_tree_data = np.load(brahma_tree_connect_file)
SubhaloIndex_mergers = brahma_tree_data["SubhaloIndex"]
FirstProgenitorID_mergers = brahma_tree_data["FirstProgenitorID"]
NextProgenitorID_mergers = brahma_tree_data["NextProgenitorID"]
DescendantID_mergers = brahma_tree_data['DescendantID']
remnant_snap_mergers =brahma_tree_data['remnant_snap']
remnant_subfindID = brahma_tree_data['SubhaloIndex']

In [55]:
remnant_subfindID

array([   -1,   263,  4119, ..., 11097,  4251,  4151])

In [61]:

worst_merger_indices[1]

32

In [62]:
np.where((SubfindID == remnant_subfindID[32])&(SnapNum == remnant_snap_mergers[32]))

(array([], dtype=int64),)

In [40]:
np.where((SubfindID==brahma_tree_data['SubhaloIndex'][2])&(SnapNum==remnant_snap_mergers[2]))

(array([728446]),)

In [10]:
# SubhaloIndex_mergers=[]
# FirstProgenitorID_mergers=[]
# NextProgenitorID_mergers=[]

# for merger_index in merger_indices:
#     subfind_ID, fpID, npID, dID, remnant_snap = get_sublink_info_for_merger_event(merger_index)
#     SubhaloIndex_mergers.append(subfind_ID)
#     FirstProgenitorID_mergers.append(fpID)
#     NextProgenitorID_mergers.append(npID)

# SubhaloIndex_mergers = np.array(SubhaloIndex_mergers)
# FirstProgenitorID_mergers = np.array(FirstProgenitorID_mergers)
# NextProgenitorID_mergers = np.array(NextProgenitorID_mergers)

In [11]:
print(len(merger_type[merger_type==1]))
print("Total mergers is ",len(NextProgenitorID_mergers))

2990
Total mergers is  3142


In [58]:
extract_proper = (FirstProgenitorID_mergers >= 0) & (NextProgenitorID_mergers >= 0)
proper_merger_indices = merger_indices[extract_proper]

extract_bad = (FirstProgenitorID_mergers >= 0) & (NextProgenitorID_mergers == -1 )
bad_merger_indices = merger_indices[extract_bad]


extract_worst = (FirstProgenitorID_mergers == -1) & (NextProgenitorID_mergers == -1 )
worst_merger_indices = merger_indices[extract_worst]


print("Number of mergers with first progenitor ID =-1 is ",len(FirstProgenitorID_mergers[extract_worst]))

print("Number of mergers with first progenitor ID != -1 but next progenitor ID = -1 is ",len(FirstProgenitorID_mergers[extract_bad]))


print("Number of mergers with neither = -1 is ",len(NextProgenitorID_mergers[proper_merger_indices]))

Number of mergers with first progenitor ID =-1 is  244
Number of mergers with first progenitor ID != -1 but next progenitor ID = -1 is  539
Number of mergers with neither = -1 is  2359


In [24]:
remnant_snap_mergers

array([16, 19, 23, ..., 26, 30, 32])

In [26]:
SnapNum

array([32, 31, 30, ..., 28, 30, 30], dtype=int16)

In [ ]:
np.where((SubfindID == merger_remnant_SubhaloID[i]) & (SnapNum == merger_remnant_snap[i]))

In [31]:
remnant_SubhaloID[worst_merger_indices[1]]

251

In [41]:
snap_bh_merger = []
snap_subhalo_merger = []

for i in worst_merger_indices:   
    fpID = FirstProgenitorID_mergers[i]
    
    tree_target_index = np.where((SubfindID==brahma_tree_data['SubhaloIndex'][i])&(SnapNum==remnant_snap_mergers[i]))[0]

    print(tree_target_index)
    if fpID==-1:
        npID = NextProgenitorID_mergers[i]
        snap_bh_merger.append(remnant_snap_mergers[i])
        #if(npID!=-1):
            #snap_subhalo_merger.append(
            
        

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [19]:
snap_bh_merger = []

for i in worst_merger_indices:   
    fpID = FirstProgenitorID_mergers[i]
    if fpID==-1:
        npID = NextProgenitorID_mergers[i]
        snap_bh_merger.append(remnant_snap_mergers[i])
        if(npID!=-1):
            
        

Desired redshift:  9.00233985416247
Output redshift:  9.00233985416247
Output snapshot:  16
16
Desired redshift:  7.005417045544533
Output redshift:  7.005417045544533
Output snapshot:  18
18
Desired redshift:  3.008131071630377
Output redshift:  3.008131071630377
Output snapshot:  22
22
Desired redshift:  2.0020281392528516
Output redshift:  2.0020281392528516
Output snapshot:  23
23
Desired redshift:  2.0020281392528516
Output redshift:  2.0020281392528516
Output snapshot:  23
23
Desired redshift:  2.0020281392528516
Output redshift:  2.0020281392528516
Output snapshot:  23
23
Desired redshift:  0.2977176845174465
Output redshift:  0.2977176845174465
Output snapshot:  29
29
Desired redshift:  4.995933468164624
Output redshift:  4.995933468164624
Output snapshot:  20
20
Desired redshift:  4.0079451114652676
Output redshift:  4.0079451114652676
Output snapshot:  21
21
Desired redshift:  4.0079451114652676
Output redshift:  4.0079451114652676
Output snapshot:  21
21
Desired redshift:  3

In [29]:
progenitor_subhalo_index, progenitor_redshift = 1619,10
desc_id,desc_snap = arepo.get_sublink_descendants(basePath,progenitor_subhalo_index, progenitor_redshift)
desc_id = np.array(desc_id)
desc_snap = np.array(desc_snap)
descendant_id = desc_id[desc_snap == remnant_snap][0]
SubhaloGrNr,o=arepo.get_subhalo_property(basePath,'SubhaloGrNr',remnant_redshift[merger_index],postprocessed=1)
SubhaloPos,o=arepo.get_subhalo_property(basePath,'SubhaloPos',remnant_redshift[merger_index],postprocessed=1)

print("--------------")
print("Sublink Descendant ID of the selected progenitor at snapshot",remnant_snap,"is ", descendant_id)
print("Remnant Subhalo ID of the merger", SubhaloIndex_mergers[merger_index])

print("Host halo of Remnant Subhalo of merger",SubhaloGrNr[SubhaloIndex_mergers[merger_index]])
print("Host halo of Sublink's Descendant subhalo",SubhaloGrNr[descendant_id])
print("If the above two match, that's great!")


print("Position of Remnant Subhalo of merger",SubhaloPos[SubhaloIndex_mergers[merger_index]])
print("Position of Sublink's Descendant subhalo",SubhaloPos[descendant_id])
print("--------------")


Desired redshift:  10
Output redshift:  9.996590466186332
Output snapshot:  15
--------------
Sublink Descendant ID of the selected progenitor at snapshot 32 is  2279
Remnant Subhalo ID of the merger 4151
Host halo of Remnant Subhalo of merger 4
Host halo of Sublink's Descendant subhalo 1
If the above two match, that's great!
Position of Remnant Subhalo of merger [12253.481    2257.422     208.86566]
Position of Sublink's Descendant subhalo [9859.758  8228.093  4306.3833]
--------------
